In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [3]:
from numpy.random import seed
from random import randrange

In [5]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)

In [6]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [60,90,120,150,180]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_head=[6,9,12,15,18]
decoder_input_size_multiplier=[0.3,0.4,0.5,0.6,0.7]
dropout=[0,0.2,0.3,0.4]
patch_len=[4,6,8,10,12]
stride=[6,7,8,9,10]
encoder_layers = [1,2,3,4,5]

# Test

In [7]:
test_length=int(len(df['NKE'])*0.3)
df1=df[int(len(df['NKE'])*0.7*0.5):].unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# H=1

In [8]:
model = PatchTST(h=1, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
              random_seed=86668740)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
if "PatchTST-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["PatchTST"].values
else:
  Y_hat=forecasts["PatchTST-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 86668740


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.003906576033788313 | QLIKE: 0.03125346710322112 | MAE: 0.0024830745349214323 | MAPE: 0.19402041207586126


In [9]:
if "PatchTST-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","PatchTST-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [10]:
df2.to_excel("PatchTST forecasts.xlsx")

# H=5

In [11]:
H=5
model = PatchTST(h=H, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
              random_seed=86668740)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "PatchTST-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["PatchTST"].values
else:
  Y_hat=forecasts["PatchTST-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "PatchTST-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","PatchTST-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 86668740


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004695863654038297 | QLIKE: 0.043744356460311586 | MAE: 0.0029557310651355286 | MAPE: 0.2306390791506471


# H=10

In [12]:
H=10
model = PatchTST(h=H, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
              random_seed=86668740)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "PatchTST-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["PatchTST"].values
else:
  Y_hat=forecasts["PatchTST-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "PatchTST-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","PatchTST-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 86668740


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.0054929272206640034 | QLIKE: 0.05604144975001364 | MAE: 0.0033709350701725703 | MAPE: 0.2570612858435188
